In [2]:
from playground.find_tps_oblique import style_config
from tests.sfm_agi.start_agi import azimuth
%load_ext autoreload
%autoreload 2

In [18]:
import src.base.connect_to_database as ctd
 
# get connection to db
conn = ctd.establish_connection()
 
# get the necessary data
sql_string = ("SELECT image_id, height, azimuth_exact, focal_length, \
              ST_ASTEXT(footprint_exact) AS footprint_exact, \
              ST_ASTEXT(position_exact) AS position_exact \
              FROM images_extracted")
data = ctd.execute_sql(sql_string, conn)
print(data.shape)

(28174, 6)


In [ ]:

PATH_GEOREF_FLD = "/data/ATM/data_1/georef/"

# load the shapefile with center data
path_center_shp = os.path.join(PATH_GEOREF_FLD, "centers",
                               georef_type + "_centers.shp")
center_shp_data = gpd.read_file(path_center_shp)


In [41]:
import shapely
import pandas as pd
import src.display.display_shapes as ds
import src.georef.snippets.calc_oblique_footprint as cof
import src.georef.snippets.calc_azimuth as ca

# shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

# iterate over the data
for row in data.iterrows():
        
    # get the row
    row = row[1]
    
    image_id = row['image_id']
    
    # skip vertical images
    if "V" in row['image_id']:
        continue
    
    # get the oblique footprint    
    oblique_wkt = row['footprint_exact']
    oblique_footprint = shapely.from_wkt(oblique_wkt)
    
    # get the oblique center
    oblique_center_wkt = row['position_exact']
    if oblique_center_wkt is None:
        continue
    oblique_center = shapely.from_wkt(oblique_center_wkt)
    
    if "31L" in row['image_id']:
        direction = "L"
    elif "33R" in row['image_id']:
        direction = "R"
    else:
        raise ValueError("Direction not found")
    
    azimuth = ca.calc_azimuth(image_id)
    
    azimuth = row['azimuth_exact']
    azimuth = azimuth + 90
    azimuth = azimuth % 360
    
    if pd.isna(azimuth):
        continue
    
    # get the vertical image for this oblique image
    vertical_id = row['image_id'].replace('31L', "32V")
    vertical_id = vertical_id.replace('33R', "32V")
    
    # get other attributes of the image
    focal_length = row['focal_length']
    altitude = row['height']
    if pd.isna(altitude):
        continue
    
    # get the vertical footprint
    vertical_wkt= data[data['image_id'] == vertical_id]['footprint_exact'].values[0]
    vertical_footprint = shapely.from_wkt(vertical_wkt)
    
    print(oblique_center)
    print(direction)
    print(focal_length)
    print(altitude)
    print(azimuth)
    
    
    # calculate the new footprint
    oblique_footprint_new = cof.calc_oblique_footprint(
        center=oblique_center, direction=direction, focal_length=focal_length, altitude=altitude, azimuth=azimuth
    )
    
    style_config={
        "title": vertical_id,
        "colors": ["blue", "red", "green"]
    }
    ds.display_shapes([vertical_footprint, oblique_footprint, oblique_footprint_new], style_config=style_config)
        
    print(f"Calc footprint for {row['image_id']}")
    

POINT (-1721560.0648188668 924570.9905799346)
L
151.776
24000.0
328.34000000000003
Start: download_rema_data (40_13)
Downloading: 100% [923572203 / 923572203] bytesFinished: download_rema_data (40_13)
Calc footprint for CA174731L0185
POINT (-1680797.0085900004 913566.8584600003)
L
155.18
25000.0
62.120000000000005
Start: download_rema_data (40_14)
Downloading: 3% [19103744 / 607617190] bytes


KeyboardInterrupt

